In [1]:
import os
import pandas as pd

In [2]:
base_dir = os.getcwd()
output_dir = os.path.join(base_dir, "..", "..", "output")

file_path1 = os.path.join(output_dir, "data_multimodal.csv")
df1 = pd.read_csv(file_path1)

In [4]:
from transformers import BertTokenizer, BertModel
import torch

C:\Users\Hp\Desktop\Data Science\My Projects\FakeNewsDetection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

In [6]:
def get_bert_embeddings(text):
    tokens = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Get CLS token embedding

In [7]:
df1.columns

Index(['Message_ID', 'Cleaned_text', 'Media File', 'Fake_News_Flag'], dtype='object')

In [8]:
df1["text_embeddings"] = df1["Cleaned_text"].apply(get_bert_embeddings)

In [9]:
df1.head()

,Message_ID,Cleaned_text,Media File,Fake_News_Flag,text_embeddings
0,1266,donald trump hate america want eliminate israe...,D:\media\israel_1266.mp4,True,"[-0.22537535, 0.4078467, 0.039324876, 0.128604..."
1,1268,breaking germany called european union classif...,NaN,True,"[-0.45706227, 0.044884853, -0.10689677, -0.231..."
2,1269,accurate photo right israel israel v hamas syr...,D:\media\israel_1269.jpg,True,"[-0.3501081, -0.12917987, -0.1392159, -0.11820..."
3,1273,breaking idf released footage precise eliminat...,D:\media\israel_1273.mp4,True,"[0.0028524054, -0.027307663, 0.12709327, 0.072..."
4,1274,love israel heart please comment shabbat shalo...,D:\media\israel_1274.jpg,True,"[-0.48694807, 0.18473333, -0.089723825, -0.521..."


In [11]:
df1.drop(columns=["Cleaned_text"], inplace=True)

In [14]:
cols = ['Message_ID', 'text_embeddings', 'Media File', 'Fake_News_Flag']

In [15]:
df1 = df1[cols]

In [18]:
file_path2 = os.path.join(output_dir, "text_multimodal.csv")
df1.to_csv(file_path2, index=False)